# YOLOv5m + CelebA

Below codes are written by me, however, 

Source code for YOLOv5 can be found at https://github.com/ultralytics/yolov5

CelebA: https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html

Some auxillary codes.

In [ ]:
def normalize(size, x1,y1,w,h):
   x = x1 + w/2
   y = y1 + h/2
   x = x/size[0]
   w = w/size[0]
   y = y/size[1]
   h = h/size[1]
   return (x,y,w,h)

In [ ]:
import os
if not os.path.exists('data'):
      os.mkdir('data')
      os.mkdir('data' + "/labels")
      os.mkdir('data' + "/images")
if not os.path.exists('valid'):
      os.mkdir('valid')
      os.mkdir('valid' + "/labels")
      os.mkdir('valid' + "/images")

Below cell copy celeba.7z from my google drive to colab.

However, if you do not have celebA dataset, please download it here:

`img_celeba.7z`: https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg?resourcekey=0-rJlzl934LzC-Xp28GeIBzQ

`list_bbox_celeba.txt`: https://drive.google.com/drive/folders/0B7EVK8r0v71pOC0wOVZlQnFfaGs?resourcekey=0-pEjrQoTrlbjZJO2UL8K_WQ

Instead of downloading it, you can use `cp` command to move to your mounted google drive /content/drive/MyDrive and unzip the `.7z` file using command below, the files will be extracted at your current directory (which is `/content`)

Note that you don't need all 200k data, For my training I used 50k and 70k data, so you can manually stop the extraction when the extracted files below is greater than 70k (or 71k if you are using 1k images for validation).

In [ ]:
! 7z x /content/drive/MyDrive/csc420/img_celeba.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/csc420/                                        1 folder, 14 files, 10191072466 bytes (9719 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/csc420/img_celeba.7z/img_celeba.7z.001
--
Path = /content/drive/MyDrive/csc420/img_celeba.7z/img_celeba.7z.001
Type = Split
Physical Size = 734003200
Volumes = 14
Total Physical Size = 10191072466
----
Path = img_celeba.7z
Size = 10191072466
--
Path = img_celeba.7z
Type = 7z
Physical Size = 10191072466
Headers Size = 1963891
Method = LZMA:24
Solid = +
Blocks = 5

  0%      0% - img_celeba/000001.jpg                              0% 5

Below 2 cells are for adding first 70k images to training set, and adding the last 1k images to validation set. According to YOLO's format.

In [ ]:
import argparse
import os
from os import walk, getcwd
from PIL import Image, ImageFile
from shutil import copyfile
import json
import requests
from io import BytesIO
ImageFile.LOAD_TRUNCATED_IMAGES = True

output_path = '/content/data'
f = open('/content/drive/MyDrive/csc420/list_bbox_celeba.txt','r')
lines = f.read().split('\n')[3:70006]
f.close()
for line in lines:
  # new_text = ""
  f = filter(None, line.split(' '))
  part = list(f)
  img_name = part[0]
  id = 0
  x = float(part[1])
  y = float(part[2])
  w = float(part[3])
  h = float(part[4])
  img_path = str(os.path.join('/content/img_celeba/','{}'.format(img_name)))
  img=Image.open(img_path)
  img.save(os.path.join('/content/data/images','%s.jpg'%(img_name.split('.')[0])))
  bb = normalize([int(img.size[0]),int(img.size[1])], x,y,w,h)
  new_text = (str(id) + " " +  " ".join([str(a) for a in bb]) + '\n')
  f = open(os.path.join('/content/data/labels','%s.txt'%(img_name.split('.')[0])),'w')
  f.write(new_text)
  f.close()


In [ ]:
output_path = '/content/valid'
f = open('/content/drive/MyDrive/csc420/list_bbox_celeba.txt','r')
lines = f.read().split('\n')[70007:71008]
f.close()
for line in lines:
  # new_text = ""
  f = filter(None, line.split(' '))
  part = list(f)
  img_name = part[0]
  id = 0
  x = float(part[1])
  y = float(part[2])
  w = float(part[3])
  h = float(part[4])
  img_path = str(os.path.join('/content/img_celeba/','{}'.format(img_name)))
  img=Image.open(img_path)
  img.save(os.path.join('/content/valid/images','%s.jpg'%(img_name.split('.')[0])))
  bb = normalize([int(img.size[0]),int(img.size[1])], x,y,w,h)
  new_text = (str(id) + " " +  " ".join([str(a) for a in bb]) + '\n')
  f = open(os.path.join('/content/valid/labels','%s.txt'%(img_name.split('.')[0])),'w')
  f.write(new_text)
  f.close()

Now we clone yolov5 here and setup the model.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12857, done.
remote: Total 12857 (delta 0), reused 0 (delta 0), pack-reused 12857
Receiving objects: 100% (12857/12857), 11.81 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (8927/8927), done.


In [ ]:
% cd yolov5/

/content/yolov5


In [ ]:
! pip install -r requirements.txt

     |████████████████████████████████| 596 kB 30.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Add below python code to a new file under yolov5/ named config.yaml**

DO NOT run below cell.

In [ ]:
##### Don't run this cell, instead copy it to a new file and name it config.yaml
path: /content/ # dataset root dir
train: data  # train images (relative to 'path') 128 images
val: valid  # val images (relative to 'path') 128 images

# Classes
nc: 1  # number of classes
names: [ 'face' ]  # class names

Now we use the config we just created above to fine-tune a yolov5 medium model for 10 epochs.

In [ ]:
! python train.py --img 640 --batch 32 --epochs 10 --data config.yaml --weights yolov5m.pt

train: weights=yolov5m.pt, cfg=, data=config.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-135-g7926afc torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5

Below code save the best weight to my google drive

In [ ]:
! cp /content/yolov5/runs/train/exp2/weights/best.pt /content/drive/MyDrive/csc420-face-crop-output

Now we use the best weight to detect faces from clip 3

In [ ]:
% cd /content

/content


In [ ]:
! wget http://www.cs.toronto.edu/~fidler/teaching/2018/CSC420_assign/project1.zip
! unzip project1.zip

In [ ]:
! python detect.py --save-txt --weights /content/yolov5/runs/train/exp2/weights/best.pt --source /content/project1/clip_3

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/project1/clip_3, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-132-g014acde torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/275 /content/project1/clip_3/0016.jpg: 384x640 2 faces, Done. (0.019s)
image 2/275 /content/project1/clip_3/0017.jpg: 384x640 2 faces, Done. (0.014s)
image 3/275 /content/project1/clip_3/0018.jpg: 384x640 2 faces, Done. (0.014s)
image 4/275 /content/project1/clip_3/0019.jpg: 384x640 2 faces, Done. (0.014s)
image 5/275 /content/pro

Below codes create a video output for detected faces in clip 3

In [ ]:
! ffmpeg -framerate 10 -start_number 16 -i /content/yolov5/runs/detect/exp2/%04d.jpg -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" output_clip3_new.mp4 

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
! zip -r /content/exp2.zip /content/yolov5/runs/train/exp2

In [ ]:
! rm /content/valid/images/*.jpg
! rm /content/valid/labels/*.txt

# MTCNN

Below is a MTCNN implementation of the face detector.

code is written by me, however the source code of MTCNN can be accessed here: https://github.com/ipazc/mtcnn

In [ ]:
import imutils
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 31.7 MB/s 


Below code detect faces in clip 1 stored at `/content/project1/clip_1`, please change this path accordingly.

In [ ]:
from mtcnn import MTCNN
import os
import time

start = time.time()
path = '/content/project1/clip_1'
outpath = './clip_1_result'
for fname in os.listdir(path):
    if ".jpg" in fname:
        img = cv2.cvtColor(cv2.imread(os.path.join(path, fname)), cv2.COLOR_BGR2RGB)
        detector = MTCNN()
        info = detector.detect_faces(img)
        for obj in info:
            xc, yc, w, h = obj['box'][0], obj['box'][1], obj['box'][2], obj['box'][3]
            xmin = int(xc)
            ymin = int(yc)
            xmax = int(xc + w)
            ymax = int(yc + h)
            img = cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 2)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(outpath, fname), img)
end = time.time()
print(end-start)

195.34867429733276


In [ ]:
! ffmpeg -framerate 10 -start_number 22 -i /content/clip_1_result/%03d.jpg -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" output_clip1.mp4 

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!zip -r clip_1_result.zip ./clip_1_result

In [ ]:
%%shell
jupyter nbconvert --to html /content/train_face_detection.ipynb

[NbConvertApp] Converting notebook /content/train_face_detection.ipynb to html
[NbConvertApp] Writing 1187093 bytes to /content/train_face_detection.html
